# Combine Data
**This notebook performs the "data cleaning" steps, as described by the Methods section of our submission.**

It also examines some of the properties of the trends data that were used in deciding the forecasting period of one month, which was described in the "Forecasting Time Frame" section of the Methods.

In [4]:
import pandas as pd
import numpy as np
from datetime import date

## Data Setup
Manipulate the song features data and Google trends data so that entries in each dataframe is indexed properly and such that only dates with entries in each dataframe are kept (i.e. if we have Google Trends data for October 11th, 2018, but we are lacking song features for that day, we should discard this data point for the purposes of modeling)

In [2]:
# Google Trends Data
individual_search_term_frequencies = pd.read_pickle('./pickles/individual_search_term_frequencies.pkl')
unscaled_relative_search_term_frequencies = pd.read_pickle('./pickles/unscaled_relative_search_term_frequencies.pkl')
scaled_relative_search_term_frequencies = pd.read_pickle('./pickles/scaled_relative_search_term_frequencies.pkl')

# Spotify data
top_200_weighted_avg = pd.read_pickle('./pickles/top_200_weighted_avg.pkl')
top_200_avg = pd.read_pickle('./pickles/top_200_avg.pkl')

# News vectors
news_vectors = pd.read_pickle('./pickles/news_vectors.pkl')

---
**Sort by date and reset indicies so that they match across dfs**

In [19]:
def sort_by_date(df):
    df.sort_values(by='date', inplace=True)
    df.reset_index(drop=True, inplace=True)

In [20]:
%%capture

sort_by_date(individual_search_term_frequencies)
sort_by_date(unscaled_relative_search_term_frequencies)
sort_by_date(scaled_relative_search_term_frequencies)
sort_by_date(top_200_weighted_avg)
sort_by_date(top_200_avg)
sort_by_date(news_vectors)

---
**Explore Data**

In [21]:
individual_search_term_frequencies.head() # scaled from 0 - 100

,date,anxiety,depression,OCD,hopeless,angry,afraid,apathy,worthless,worried,restless,irritable,tense,scattered,tired,avoiding,procrastinate,insomnia,suicidal,suicide
0,2017-01-01,73.0,65.0,73.0,58.0,99.0,58.0,52.0,52.0,57.0,59.0,71.0,46.0,40.0,68.0,50.0,64.0,71.0,84.0,100.0
1,2017-01-02,84.0,73.0,69.0,68.0,88.0,55.0,53.0,46.0,72.0,97.0,81.0,59.0,54.0,68.0,47.0,68.0,82.0,93.0,94.0
2,2017-01-03,90.0,75.0,71.0,63.0,79.0,54.0,72.0,76.0,80.0,96.0,80.0,69.0,44.0,71.0,63.0,65.0,89.0,85.0,78.0
3,2017-01-04,88.0,78.0,77.0,54.0,84.0,52.0,67.0,69.0,82.0,97.0,80.0,79.0,53.0,75.0,61.0,56.0,87.0,87.0,76.0
4,2017-01-05,91.0,78.0,72.0,66.0,84.0,54.0,65.0,51.0,64.0,81.0,81.0,81.0,71.0,70.0,60.0,57.0,85.0,86.0,78.0


In [22]:
unscaled_relative_search_term_frequencies.head() # scaled arbitrarily so that all term scores are comparable

,date,depression,anxiety,OCD,hopeless,angry,afraid,apathy,worthless,worried,...,irritable,tense,scattered,tired,avoiding,procrastinate,insomnia,suicidal,suicide,Term Scores Summed
0,2017-01-01,62.0,62.0,13.0,2.0,78.0,17.714286,1.968254,1.968254,3.936508,...,4.920635,21.650794,0.984127,26.303030,1.878788,0.939394,14.090909,6.526316,155.000000,491.611453
1,2017-01-02,69.0,70.0,13.0,3.0,69.0,17.742857,2.957143,0.985714,4.928571,...,5.914286,24.642857,1.971429,29.000000,3.000000,1.000000,18.000000,4.928571,144.571429,507.300000
2,2017-01-03,78.0,74.0,12.0,2.0,62.0,18.233766,3.038961,1.012987,5.064935,...,6.077922,29.376623,1.012987,29.376623,4.051948,1.012987,19.246753,5.086957,123.782609,499.700734
3,2017-01-04,78.0,76.0,14.0,2.0,67.0,18.233766,3.038961,2.025974,5.064935,...,7.090909,34.441558,2.025974,33.729730,3.162162,2.108108,20.027027,6.782609,125.478261,524.521663
4,2017-01-05,78.0,76.0,13.0,3.0,65.0,17.772152,2.962025,1.974684,3.949367,...,5.924051,34.556962,1.974684,32.582278,2.962025,1.974684,17.772152,6.638298,129.446809,514.249663


In [23]:
scaled_relative_search_term_frequencies.head() # scaled from 0-1000, where all term scores are comparable

,date,depression,anxiety,OCD,hopeless,angry,afraid,apathy,worthless,worried,...,irritable,tense,scattered,tired,avoiding,procrastinate,insomnia,suicidal,suicide,Term Scores Summed
0,2017-01-01,166.0,166.0,35.0,5.0,209.0,48.0,5.0,5.0,11.0,...,13.0,59.0,3.0,70.0,5.0,3.0,38.0,19.0,416.0,1322.0
1,2017-01-02,185.0,188.0,35.0,8.0,185.0,48.0,8.0,3.0,13.0,...,16.0,67.0,5.0,78.0,8.0,3.0,48.0,13.0,389.0,1364.0
2,2017-01-03,209.0,198.0,32.0,5.0,166.0,48.0,8.0,3.0,13.0,...,16.0,78.0,3.0,78.0,11.0,3.0,51.0,13.0,332.0,1334.0
3,2017-01-04,209.0,204.0,38.0,5.0,180.0,48.0,8.0,5.0,13.0,...,19.0,91.0,5.0,91.0,8.0,5.0,54.0,19.0,335.0,1401.0
4,2017-01-05,209.0,204.0,35.0,8.0,174.0,48.0,8.0,5.0,11.0,...,16.0,94.0,5.0,88.0,8.0,5.0,48.0,19.0,346.0,1382.0


In [24]:
top_200_avg.head() # all of top 200 songs are averaged, regardless of song popularity

,date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,2017-01-01,0.681935,0.631440,5.345,-6.834550,0.595,0.133264,0.174817,0.010311,0.172115,0.507134,123.711640,219318.020,4.010
1,2017-01-02,0.676290,0.610385,5.215,-6.780710,0.560,0.141241,0.198055,0.007679,0.159602,0.442370,121.723755,223566.330,3.985
2,2017-01-03,0.674705,0.608545,5.235,-6.810915,0.555,0.140566,0.198139,0.007680,0.155095,0.439146,121.517035,222978.725,3.985
3,2017-01-04,0.675600,0.606130,5.215,-6.836510,0.555,0.142691,0.202651,0.007679,0.156017,0.437445,121.867565,222926.035,3.985
4,2017-01-05,0.678950,0.609420,5.240,-6.769335,0.560,0.146080,0.199928,0.007648,0.158186,0.442190,122.244085,223008.005,3.985


In [25]:
top_200_weighted_avg.head() # the weighted average of the songs in the top 200,
                            # where the weights are the number of streams per song

,date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration,time_signature
0,2017-01-01,0.700672,0.620866,5.516916,-6.817967,0.618192,0.148352,0.175407,0.007983,0.166109,0.489005,124.906114,3.743462,4.010772
1,2017-01-02,0.694911,0.603411,5.401974,-6.817318,0.599963,0.154474,0.192934,0.006603,0.157463,0.440288,123.110631,3.780848,3.995896
2,2017-01-03,0.692630,0.602649,5.415236,-6.824104,0.596881,0.153246,0.194011,0.006792,0.154835,0.438192,122.733971,3.774221,3.995294
3,2017-01-04,0.692878,0.601787,5.338150,-6.832330,0.586901,0.154600,0.196601,0.006914,0.155729,0.437437,122.894072,3.772473,3.994701
4,2017-01-05,0.695205,0.602774,5.399846,-6.798211,0.598931,0.156652,0.196215,0.006865,0.155725,0.439196,123.133659,3.775106,3.994540


In [26]:
news_vectors.head() # 512 dimensional vectors representing the news on a given day

,date,0,1,2,3,4,5,6,7,8,...,502,503,504,505,506,507,508,509,510,511
0,2017-01-01,0.005954,-0.003841,-0.001854,-0.024854,-0.020176,0.011049,-0.004546,-0.001006,0.005869,...,0.003684,0.035374,0.016044,-0.000216,0.023061,-0.012140,-0.002986,-0.013999,-0.011632,0.008429
1,2017-01-02,0.000123,-0.025581,-0.003911,-0.019659,-0.016987,0.012803,0.007522,-0.009581,0.009210,...,-0.000628,0.020751,0.000568,0.004707,0.005168,-0.004014,-0.007200,0.003860,-0.017291,0.009441
2,2017-01-03,0.007024,-0.021057,0.000409,-0.025842,-0.016621,0.007097,0.000726,-0.007290,0.000116,...,-0.000661,0.030381,0.013288,0.005367,0.010509,-0.006045,-0.008171,0.003855,-0.007096,0.018201
3,2017-01-04,0.004653,-0.012411,-0.001151,-0.026045,-0.011844,0.010583,0.000069,-0.003124,0.006726,...,0.004361,0.025416,0.006094,0.008971,0.008110,0.000185,-0.012961,0.004362,-0.004057,0.010306
4,2017-01-05,0.003516,-0.004221,0.005870,-0.024142,-0.009221,0.011480,0.009126,-0.000250,0.004909,...,0.010125,0.020535,0.008477,0.003254,0.008739,-0.000438,-0.007905,0.002326,0.000348,0.014819


---
**Remove any entries which don't have a entries in all tables** (some dates are missing in both spotify data 
due to service outages and data collection problems experienced by Spotify or our data provider for Spotify,
Spotify Charts)

In [27]:
entries_in_all_dfs = pd.merge(left=scaled_relative_search_term_frequencies, 
                              right=unscaled_relative_search_term_frequencies, on='date')

entries_in_all_dfs = pd.merge(left=entries_in_all_dfs, 
                              right=individual_search_term_frequencies, on='date')

entries_in_all_dfs = pd.merge(left=entries_in_all_dfs, 
                              right=top_200_avg, on='date')

entries_in_all_dfs = pd.merge(left=entries_in_all_dfs, 
                              right=top_200_weighted_avg, on='date')

entries_in_all_dfs = pd.merge(left=entries_in_all_dfs, 
                              right=news_vectors, on='date')


shared_dates = entries_in_all_dfs['date']

In [28]:
# Keep only entries with dates that exist in all 3 dataframes
def keep_shared_dates(df, shared_dates):
    return df[df['date'].isin(shared_dates)]
    
individual_search_term_frequencies = keep_shared_dates(individual_search_term_frequencies, shared_dates)
unscaled_relative_search_term_frequencies = keep_shared_dates(unscaled_relative_search_term_frequencies, shared_dates)
scaled_relative_search_term_frequencies = keep_shared_dates(scaled_relative_search_term_frequencies, shared_dates)
top_200_weighted_avg = keep_shared_dates(top_200_weighted_avg, shared_dates)
top_200_avg = keep_shared_dates(top_200_avg, shared_dates)
news_vectors = keep_shared_dates(news_vectors, shared_dates)

In [29]:
individual_search_term_frequencies.reset_index(drop=True, inplace=True)
unscaled_relative_search_term_frequencies.reset_index(drop=True, inplace=True)
scaled_relative_search_term_frequencies.reset_index(drop=True, inplace=True)
top_200_weighted_avg.reset_index(drop=True, inplace=True)
top_200_avg.reset_index(drop=True, inplace=True)
news_vectors.reset_index(drop=True, inplace=True)

## Save data

In [15]:
individual_search_term_frequencies.to_pickle('./pickles/individual_search_term_frequencies.pkl')
unscaled_relative_search_term_frequencies.to_pickle('./pickles/unscaled_relative_search_term_frequencies.pkl')
scaled_relative_search_term_frequencies.to_pickle('./pickles/scaled_relative_search_term_frequencies.pkl')
top_200_weighted_avg.to_pickle('./pickles/top_200_weighted_avg.pkl')
top_200_avg.to_pickle('./pickles/top_200_avg.pkl')
news_vectors.to_pickle('./pickles/news_vectors.pkl')

## Explore Properties of the Data

In [5]:
# Show number of missing dates
start = date(2017, 1, 1)
end = date(2020, 9, 29)
delta = end - start
print('Number of missing days:', delta.days - len(individual_search_term_frequencies))

Number of missing days: 3


In [6]:
# Show total number of training samples
len(individual_search_term_frequencies)

1364

In [7]:
def get_avg_diff(term_list):
    diff_list = []
    for x, y in zip(term_list[0:], term_list[1:]):
        diff_list.append(np.abs(y-x))

    print(sum(diff_list) / len(diff_list))

In [8]:
# Show how search term scores differ from day-to-day
daily_list = individual_search_term_frequencies['anxiety'].to_list()
get_avg_diff(daily_list)

5.7921537132862815


In [9]:
# Show how search term scores differ from month-to-month
monthly_list = []
for i in range(30, len(daily_list) + 1, 30):
    monthly_list.append(sum(daily_list[i-30: i]) / 30)
get_avg_diff(monthly_list)

4.1162468404682135


In [10]:
# Show how search term scores differ from year-to-year
yearly_list = []
for i in range(365, len(daily_list) + 1, 365):
    yearly_list.append(sum(daily_list[i-365: i]) / 365)
get_avg_diff(yearly_list)

8.693062156491749


In [12]:
# Show how search term scores differ from day-to-day
daily_list = individual_search_term_frequencies['depression'].to_list()
get_avg_diff(daily_list)

5.834378803156963


In [13]:
# Show how search term scores differ from month-to-month
monthly_list = []
for i in range(30, len(daily_list) + 1, 30):
    monthly_list.append(sum(daily_list[i-30: i]) / 30)
get_avg_diff(monthly_list)

4.3305117530044495


In [14]:
# Show how search term scores differ from year-to-year
yearly_list = []
for i in range(365, len(daily_list) + 1, 365):
    yearly_list.append(sum(daily_list[i-365: i]) / 365)
get_avg_diff(yearly_list)

0.7820074712678036
